**HYPOTHESIS TESTING**

In [44]:
# IMPORT LIBRARIES
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from utils.utils import *

***z statistic** is used when using one sample statistic to estimate a population parameter*

***t statistic** is used when using multiple sample statistic to estimate a population parameter*

In [45]:
# Read the data
data = pd.read_feather("stack_overflow.feather")
# show the first 5 rows
# print(data.head())

# print column names
columns = list(data.columns)
print(columns)

# Filter columns that contain 'age'
# columns = [col for col in columns if 'age' in col]
columns = search_string_in_columns(df=data, search_string='age')

print(columns)

['respondent', 'main_branch', 'hobbyist', 'age', 'age_1st_code', 'age_first_code_cut', 'comp_freq', 'comp_total', 'converted_comp', 'country', 'currency_desc', 'currency_symbol', 'database_desire_next_year', 'database_worked_with', 'dev_type', 'ed_level', 'employment', 'ethnicity', 'gender', 'job_factors', 'job_sat', 'job_seek', 'language_desire_next_year', 'language_worked_with', 'misc_tech_desire_next_year', 'misc_tech_worked_with', 'new_collab_tools_desire_next_year', 'new_collab_tools_worked_with', 'new_dev_ops', 'new_dev_ops_impt', 'new_ed_impt', 'new_job_hunt', 'new_job_hunt_research', 'new_learn', 'new_off_topic', 'new_onboard_good', 'new_other_comms', 'new_overtime', 'new_purchase_research', 'purple_link', 'newso_sites', 'new_stuck', 'op_sys', 'org_size', 'platform_desire_next_year', 'platform_worked_with', 'purchase_what', 'sexuality', 'so_account', 'so_comm', 'so_part_freq', 'so_visit_freq', 'survey_ease', 'survey_length', 'trans', 'undergrad_major', 'webframe_desire_next_yea

**SCORE Z: COMPARE SAMPLE STATISTICS ACCROS ON GROUP OF VARIABLE**

In [46]:
# Calculate the mean and standard deviation of the salary
col = 'converted_comp'
mean_salary = data[col].mean()
std_salary = data[col].std()
print(f"Mean salary: {mean_salary}")
print(f"Standard deviation of salary: {std_salary}")

# Calculate Z-scores
data["z_score"] = (data[col] - mean_salary) / std_salary
# print the first 5 rows
print(data["z_score"].head())


Mean salary: 119574.71738168952
Standard deviation of salary: 265556.0729116986
0   -0.158229
1   -0.167967
2    1.788565
3    7.081086
4   -0.307877
Name: z_score, dtype: float64


In [47]:
num_col = 'converted_comp'
cat_col = 'age_first_code_cut'
data.select_dtypes('category').columns
# Convert the categorical column to a category type
data[cat_col] = data[cat_col].astype('category')

data.select_dtypes('category').columns

Index(['age_first_code_cut', 'job_sat'], dtype='object')

**TEST T: COMPARE SAMPLE STATISTICS ACCROS GROUPS OF VARIABLE (2 GROUPS)**

In [48]:
# ARE USERS WHO FIRST PROGRAMMED AS CHILD COMPENSATED HIGHER THAN THOSE THAT STARTED AS ADULTS ?

# NULL HYPOTHESIS: The mean compensation is the same for those who started programming as children and those who started as adults.
# ALTERNATIVE HYPOTHESIS: The mean compensation is greater than the mean for those who started programming as children and those who started as adults.
# 


In [49]:
# age_first_code_cut summary statistics
age_first_code_cut = data[cat_col].value_counts()
print(age_first_code_cut)

draw_stars_line()

# mean salary for each group
mean_salary = data.groupby(cat_col, observed=True)[num_col].mean()
print(mean_salary)

age_first_code_cut
adult    1376
child     885
Name: count, dtype: int64


**************************************************

age_first_code_cut
adult    111313.311047
child    132419.570621
Name: converted_comp, dtype: float64


In [50]:
mean_salary = data.groupby(cat_col, observed=True)[num_col].mean().reset_index()
print(mean_salary)
draw_stars_line()
std_salary = data.groupby(cat_col, observed=True)[num_col].std().reset_index()
# count_group = data.groupby(cat_col, observed=True)[num_col].count()
print(std_salary)
draw_stars_line()

count_group = data[cat_col].value_counts().reset_index()
print(count_group)

print(count_group.iloc[0,1])



  age_first_code_cut  converted_comp
0              adult   111313.311047
1              child   132419.570621


**************************************************

  age_first_code_cut  converted_comp
0              adult   271546.521729
1              child   255585.240115


**************************************************

  age_first_code_cut  count
0              adult   1376
1              child    885
1376


In [51]:
# Calculate T statistic
# T statistic = (mean1 - mean2) / sqrt((std1^2/n1) + (std2^2/n2))

numerator = mean_salary.iloc[1,1] - mean_salary.iloc[0,1]
print(numerator)
draw_stars_line()

denominator = np.sqrt((std_salary.iloc[0,1]**2/count_group.iloc[0,1]) + (std_salary.iloc[1,1]**2/count_group.iloc[1,1]))

print(denominator)
draw_stars_line()

t_statistic = numerator / denominator
print("t statistic is: ", t_statistic)

# Calculate the degrees of freedom
# degrees of freedom = n1 + n2 - 2
df = count_group.iloc[0,1] + count_group.iloc[1,1] - 2
print("degrees of freedom is: ", df)

# Calculate the critical value
# critical value = t.ppf(1 - alpha, df)
from scipy.stats import t
alpha = 0.05


# Calculate the p-value
# p-value = 1 - t.cdf(t_statistic, df)
p_value = 1 - t.cdf(t_statistic, df) #cdf = cumulative distribution function
print("p-value is: ", p_value)

# Compare the p-value to the significance level
if p_value < alpha:
    print("Reject the null hypothesis")
else:
    print("Fail to reject the null hypothesis")

21106.25957495728


**************************************************

11287.18430352594


**************************************************

t statistic is:  1.8699313316221844
degrees of freedom is:  2259
p-value is:  0.030811302165157595
Reject the null hypothesis


**PAIRED T TEST: COMPARE SAMPLE STATISTICS ACCROS MULTIPLE GROUP OF VARIABLE**

In [53]:
# Import dataset
repub08_12 = pd.read_feather('repub_votes_potus_08_12.feather')
dem_12_16 = pd.read_feather('dem_votes_potus_12_16.feather')

# Show the first 5 rows
print(repub08_12.head())
draw_stars_line()
print(dem_12_16.head())

        state   county  repub_percent_08  repub_percent_12
0     Alabama     Hale         38.957877         37.139882
1    Arkansas   Nevada         56.726272         58.983452
2  California     Lake         38.896719         39.331367
3  California  Ventura         42.923190         45.250693
4    Colorado  Lincoln         74.522569         73.764757


**************************************************

     state    county  dem_percent_12  dem_percent_16
0  Alabama   Bullock       76.305900       74.946921
1  Alabama   Chilton       19.453671       15.847352
2  Alabama      Clay       26.673672       18.674517
3  Alabama   Cullman       14.661752       10.028252
4  Alabama  Escambia       36.915731       31.020546
